# Pitfall 03: DID 平行趋势违背与诊断

## 🎯 学习目标

完成本 notebook 后，你将能够：

1. **理解** DID 平行趋势假设的本质
2. **诊断** 平行趋势是否满足
3. **识别** Anticipation Effect 和其他违背情形
4. **应用** 替代方法处理平行趋势违背

---

## 📖 背景故事

> **场景：新功能上线效果评估**
>
> 你负责评估一个新功能对用户活跃度的影响。由于无法做 A/B 测试（功能已全量上线），
> 你决定用 DID（双重差分）方法，比较上线前后处理组和对照组的变化。
>
> 但老板质疑："你怎么证明没有这个功能，处理组和对照组的趋势会一样？"
>
> 更糟的是，你发现处理组在功能上线**前一周**活跃度就开始上升了...
>
> **这就是 Anticipation Effect！本 notebook 将教你如何诊断和处理。**

---

## Part 1: DID 核心假设回顾

### 1.1 DID 估计量

$$\hat{\tau}_{DID} = (\bar{Y}_{T,post} - \bar{Y}_{T,pre}) - (\bar{Y}_{C,post} - \bar{Y}_{C,pre})$$

### 1.2 平行趋势假设 (Parallel Trends)

**核心假设**：在没有处理的情况下，处理组和对照组的结果变量会以相同的趋势变化。

$$E[Y_{it}(0) - Y_{is}(0) | D_i = 1] = E[Y_{it}(0) - Y_{is}(0) | D_i = 0]$$

对于所有 $t > s$（处理前的任意两个时期）。

### 1.3 为什么这个假设重要？

如果平行趋势不满足：
- DID 估计量有偏
- 无法分离处理效应和自然趋势差异

In [ ]:
# 环境准备
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = ['Arial Unicode MS', 'SimHei', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 环境准备完成")

In [ ]:
def generate_did_data(n_units=200, n_periods=12, treatment_period=6,
                       parallel=True, anticipation=False, true_effect=5.0, seed=42):
    """
    生成 DID 模拟数据
    
    Args:
        n_units: 个体数量
        n_periods: 时间期数
        treatment_period: 处理开始的期数
        parallel: 是否满足平行趋势
        anticipation: 是否有预期效应
        true_effect: 真实处理效应
    """
    np.random.seed(seed)
    
    # 生成个体和时间
    units = np.repeat(np.arange(n_units), n_periods)
    periods = np.tile(np.arange(n_periods), n_units)
    
    # 处理组分配（前一半是处理组）
    treated = (units < n_units // 2).astype(int)
    
    # 处理后指示变量
    post = (periods >= treatment_period).astype(int)
    
    # 基础趋势
    if parallel:
        # 满足平行趋势：两组相同的时间趋势
        time_trend = 0.5 * periods
    else:
        # 不满足平行趋势：处理组有不同的趋势
        time_trend = 0.5 * periods + 0.3 * periods * treated
    
    # 个体固定效应
    unit_fe = np.random.randn(n_units)[units] * 3
    
    # 组别基础差异
    group_effect = 2 * treated
    
    # 处理效应
    treatment_effect = true_effect * treated * post
    
    # Anticipation Effect
    if anticipation:
        # 处理前 2 期开始有预期效应
        anticipation_periods = (periods >= treatment_period - 2) & (periods < treatment_period)
        anticipation_effect = 2 * treated * anticipation_periods
    else:
        anticipation_effect = 0
    
    # 噪声
    noise = np.random.randn(len(units)) * 2
    
    # 结果变量
    Y = 10 + unit_fe + group_effect + time_trend + treatment_effect + anticipation_effect + noise
    
    return pd.DataFrame({
        'unit': units,
        'period': periods,
        'treated': treated,
        'post': post,
        'Y': Y
    })

# 生成三种场景的数据
df_parallel = generate_did_data(parallel=True, anticipation=False)
df_nonparallel = generate_did_data(parallel=False, anticipation=False)
df_anticipation = generate_did_data(parallel=True, anticipation=True)

print("生成三种场景数据：")
print(f"  1. 满足平行趋势: {len(df_parallel)} 观测")
print(f"  2. 不满足平行趋势: {len(df_nonparallel)} 观测")
print(f"  3. 有 Anticipation Effect: {len(df_anticipation)} 观测")

In [ ]:
# 可视化三种场景
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

scenarios = [
    (df_parallel, '满足平行趋势', 'green'),
    (df_nonparallel, '不满足平行趋势', 'red'),
    (df_anticipation, 'Anticipation Effect', 'orange')
]

for ax, (df, title, color) in zip(axes, scenarios):
    # 计算每期每组的均值
    means = df.groupby(['period', 'treated'])['Y'].mean().unstack()
    
    ax.plot(means.index, means[0], 'b-o', label='对照组', linewidth=2)
    ax.plot(means.index, means[1], 'r-o', label='处理组', linewidth=2)
    ax.axvline(x=5.5, color='gray', linestyle='--', label='处理时点')
    ax.fill_betweenx([means.values.min()-1, means.values.max()+1], 6, 11, 
                     alpha=0.2, color='gray')
    ax.set_xlabel('时期')
    ax.set_ylabel('结果 Y')
    ax.set_title(title)
    ax.legend(loc='upper left')

plt.tight_layout()
plt.show()

print("\n观察：")
print("  左图：处理前两组趋势平行，DID 有效")
print("  中图：处理前处理组趋势更陡，DID 会高估效应")
print("  右图：处理前处理组已开始上升 (Anticipation)，需要调整")

---

## Part 2: 平行趋势检验

### 2.1 可视化检验

最直观的方法：画图看处理前两组的趋势是否平行。

### 2.2 事件研究法 (Event Study)

更正式的检验：估计每个时期的"处理效应"，处理前的效应应该接近 0。

$$Y_{it} = \alpha_i + \gamma_t + \sum_{k \neq -1} \beta_k \cdot D_i \cdot \mathbb{1}[t = k] + \epsilon_{it}$$

其中 $k = -1$（处理前一期）作为基准。

In [ ]:
def event_study(df, treatment_period=6):
    """
    事件研究法检验平行趋势
    
    Returns:
        coefficients: 每期的系数估计
        ci_lower, ci_upper: 95% 置信区间
    """
    # 创建相对时间变量
    df = df.copy()
    df['rel_time'] = df['period'] - treatment_period
    
    # 创建交互项（排除 -1 期作为基准）
    rel_times = sorted(df['rel_time'].unique())
    rel_times.remove(-1)  # 基准期
    
    for t in rel_times:
        df[f'D_t{t}'] = (df['treated'] == 1) & (df['rel_time'] == t)
        df[f'D_t{t}'] = df[f'D_t{t}'].astype(int)
    
    # 构建公式
    interaction_terms = ' + '.join([f'D_t{t}' for t in rel_times])
    formula = f'Y ~ C(unit) + C(period) + {interaction_terms}'
    
    # 回归
    model = smf.ols(formula, data=df).fit(cov_type='cluster', 
                                           cov_kwds={'groups': df['unit']})
    
    # 提取系数
    results = {'rel_time': [-1], 'coef': [0], 'ci_lower': [0], 'ci_upper': [0]}
    
    for t in rel_times:
        param_name = f'D_t{t}[T.True]'
        if param_name in model.params:
            coef = model.params[param_name]
            se = model.bse[param_name]
            results['rel_time'].append(t)
            results['coef'].append(coef)
            results['ci_lower'].append(coef - 1.96 * se)
            results['ci_upper'].append(coef + 1.96 * se)
    
    return pd.DataFrame(results).sort_values('rel_time')

# 对三种场景进行事件研究
es_parallel = event_study(df_parallel)
es_nonparallel = event_study(df_nonparallel)
es_anticipation = event_study(df_anticipation)

print("事件研究结果计算完成")

In [ ]:
# 可视化事件研究结果
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

event_studies = [
    (es_parallel, '满足平行趋势', 'green'),
    (es_nonparallel, '不满足平行趋势', 'red'),
    (es_anticipation, 'Anticipation Effect', 'orange')
]

for ax, (es, title, color) in zip(axes, event_studies):
    # 分离处理前后
    pre = es[es['rel_time'] < 0]
    post = es[es['rel_time'] >= 0]
    
    # 画处理前
    ax.errorbar(pre['rel_time'], pre['coef'], 
               yerr=[pre['coef'] - pre['ci_lower'], pre['ci_upper'] - pre['coef']],
               fmt='o-', color='blue', capsize=4, label='处理前')
    
    # 画处理后
    ax.errorbar(post['rel_time'], post['coef'], 
               yerr=[post['coef'] - post['ci_lower'], post['ci_upper'] - post['coef']],
               fmt='s-', color='red', capsize=4, label='处理后')
    
    ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
    ax.axvline(x=-0.5, color='black', linestyle='-', alpha=0.3)
    ax.fill_betweenx(ax.get_ylim(), -0.5, ax.get_xlim()[1], alpha=0.1, color='red')
    ax.set_xlabel('相对时间 (处理时点 = 0)')
    ax.set_ylabel('估计系数')
    ax.set_title(title)
    ax.legend()

plt.tight_layout()
plt.show()

print("\n解读：")
print("  ✅ 左图：处理前系数都接近 0，平行趋势成立")
print("  ❌ 中图：处理前系数呈上升趋势，平行趋势不成立")
print("  ⚠️ 右图：处理前 t=-2 和 t=-1 期系数显著为正，存在 Anticipation")

### 2.3 平行趋势的统计检验

In [ ]:
def parallel_trend_test(df, treatment_period=6):
    """
    平行趋势的正式统计检验
    
    H0: 处理前所有交互项系数 = 0（平行趋势成立）
    """
    # 只使用处理前数据
    df_pre = df[df['period'] < treatment_period].copy()
    
    # 检验处理组*时间的交互效应
    df_pre['treated_time'] = df_pre['treated'] * df_pre['period']
    
    # 回归模型
    formula = 'Y ~ C(unit) + C(period) + treated_time'
    model = smf.ols(formula, data=df_pre).fit(
        cov_type='cluster', cov_kwds={'groups': df_pre['unit']}
    )
    
    # 提取交互项系数
    coef = model.params['treated_time']
    se = model.bse['treated_time']
    t_stat = coef / se
    p_value = 2 * (1 - stats.t.cdf(abs(t_stat), df=model.df_resid))
    
    return {
        'coefficient': coef,
        'se': se,
        't_stat': t_stat,
        'p_value': p_value,
        'reject_H0': p_value < 0.05
    }

# 对三种场景进行检验
print("=" * 60)
print("平行趋势检验结果")
print("=" * 60)
print("H0: 处理前处理组和对照组趋势平行")
print("\n")

for name, df in [('满足平行趋势', df_parallel), 
                  ('不满足平行趋势', df_nonparallel),
                  ('Anticipation Effect', df_anticipation)]:
    result = parallel_trend_test(df)
    status = '❌ 拒绝 H0' if result['reject_H0'] else '✅ 不拒绝 H0'
    print(f"{name}:")
    print(f"  交互项系数: {result['coefficient']:.4f}")
    print(f"  t 统计量: {result['t_stat']:.3f}")
    print(f"  p 值: {result['p_value']:.4f}")
    print(f"  结论: {status}")
    print()

---

## Part 3: Anticipation Effect 的诊断与处理

### 3.1 什么是 Anticipation Effect?

**定义**：在处理正式实施前，处理组已经开始改变行为。

**常见原因**：
- 政策提前公布
- 信息泄露
- 渐进式实施

**后果**：
- 标准 DID 低估真实效应
- 平行趋势检验可能通过（如果只看更早的时期）

In [ ]:
def diagnose_anticipation(df, treatment_period=6, n_pre_periods=3):
    """
    诊断 Anticipation Effect
    
    检查处理前最后几期是否有显著效应
    """
    es = event_study(df, treatment_period)
    
    # 检查处理前最后几期
    pre_effects = es[(es['rel_time'] < 0) & (es['rel_time'] >= -n_pre_periods)]
    
    print("Anticipation Effect 诊断")
    print("=" * 50)
    print(f"\n检查处理前 {n_pre_periods} 期的系数：")
    print()
    
    has_anticipation = False
    for _, row in pre_effects.iterrows():
        t = int(row['rel_time'])
        sig = '***' if (row['ci_lower'] > 0 or row['ci_upper'] < 0) else ''
        if sig:
            has_anticipation = True
        print(f"  t = {t}: 系数 = {row['coef']:.3f}, 95% CI = [{row['ci_lower']:.3f}, {row['ci_upper']:.3f}] {sig}")
    
    print()
    if has_anticipation:
        print("⚠️ 警告: 检测到 Anticipation Effect!")
        print("   建议: 调整处理时点或使用调整后的 DID")
    else:
        print("✅ 未检测到显著的 Anticipation Effect")
    
    return has_anticipation

# 诊断
print("\n--- 场景: 有 Anticipation Effect ---")
diagnose_anticipation(df_anticipation)

print("\n\n--- 场景: 满足平行趋势 ---")
diagnose_anticipation(df_parallel)

### 3.2 处理 Anticipation Effect

**方法 1: 调整处理时点**

将处理时点提前到 Anticipation 开始的时候。

In [ ]:
def did_estimate(df, treatment_period=6):
    """
    标准 DID 估计
    """
    df = df.copy()
    df['post'] = (df['period'] >= treatment_period).astype(int)
    df['did'] = df['treated'] * df['post']
    
    formula = 'Y ~ C(unit) + C(period) + did'
    model = smf.ols(formula, data=df).fit(
        cov_type='cluster', cov_kwds={'groups': df['unit']}
    )
    
    coef = model.params['did']
    se = model.bse['did']
    
    return coef, se

# 比较不同处理时点的 DID 估计
print("处理 Anticipation Effect：调整处理时点")
print("=" * 50)
print(f"真实处理效应 = 5.0")
print()

treatment_periods_to_try = [4, 5, 6, 7]

for tp in treatment_periods_to_try:
    coef, se = did_estimate(df_anticipation, treatment_period=tp)
    print(f"处理时点 = {tp}: DID 估计 = {coef:.3f} (SE = {se:.3f})")

print()
print("✅ 将处理时点从 6 调整到 4 可以更好地捕捉全部效应")

---

## Part 4: 平行趋势不满足时的替代方法

### 4.1 控制组特定趋势 (Group-Specific Trends)

In [ ]:
def did_with_group_trends(df, treatment_period=6):
    """
    带有组特定趋势的 DID
    
    允许处理组和对照组有不同的线性趋势
    """
    df = df.copy()
    df['post'] = (df['period'] >= treatment_period).astype(int)
    df['did'] = df['treated'] * df['post']
    df['treated_trend'] = df['treated'] * df['period']
    
    formula = 'Y ~ C(unit) + C(period) + treated_trend + did'
    model = smf.ols(formula, data=df).fit(
        cov_type='cluster', cov_kwds={'groups': df['unit']}
    )
    
    return {
        'did_coef': model.params['did'],
        'did_se': model.bse['did'],
        'trend_coef': model.params['treated_trend'],
        'trend_se': model.bse['treated_trend']
    }

# 对不满足平行趋势的数据应用
print("组特定趋势 DID (用于不满足平行趋势的情况)")
print("=" * 50)
print(f"真实处理效应 = 5.0\n")

# 标准 DID
standard_coef, standard_se = did_estimate(df_nonparallel)
print(f"标准 DID: {standard_coef:.3f} (SE = {standard_se:.3f})")

# 组特定趋势 DID
result = did_with_group_trends(df_nonparallel)
print(f"组特定趋势 DID: {result['did_coef']:.3f} (SE = {result['did_se']:.3f})")
print(f"  处理组额外趋势: {result['trend_coef']:.3f}/期")

print()
print("✅ 组特定趋势 DID 更接近真实效应 5.0")

### 4.2 合成控制法 (Synthetic Control)

当只有一个或少数处理单位时，可以用合成控制法构造"反事实"。

In [ ]:
def simple_synthetic_control(df, treated_unit, treatment_period=6):
    """
    简化版合成控制法
    
    用对照组的加权平均来拟合处理单位的处理前趋势
    """
    from scipy.optimize import minimize
    
    # 分离数据
    df_pre = df[df['period'] < treatment_period]
    df_post = df[df['period'] >= treatment_period]
    
    # 处理单位的处理前结果
    y_treated_pre = df_pre[df_pre['unit'] == treated_unit].sort_values('period')['Y'].values
    
    # 对照单位的处理前结果
    control_units = df[df['treated'] == 0]['unit'].unique()
    Y_control_pre = np.array([
        df_pre[df_pre['unit'] == u].sort_values('period')['Y'].values
        for u in control_units
    ])
    
    # 优化权重
    n_controls = len(control_units)
    
    def objective(w):
        synthetic = Y_control_pre.T @ w
        return np.sum((y_treated_pre - synthetic) ** 2)
    
    # 约束：权重和为 1，非负
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1)] * n_controls
    
    # 初始值
    w0 = np.ones(n_controls) / n_controls
    
    result = minimize(objective, w0, method='SLSQP', 
                     bounds=bounds, constraints=constraints)
    
    weights = result.x
    
    # 计算处理效应
    Y_control_post = np.array([
        df_post[df_post['unit'] == u].sort_values('period')['Y'].values
        for u in control_units
    ])
    
    y_treated_post = df_post[df_post['unit'] == treated_unit].sort_values('period')['Y'].values
    y_synthetic_post = Y_control_post.T @ weights
    
    treatment_effect = y_treated_post - y_synthetic_post
    
    return {
        'weights': weights,
        'treatment_effect': treatment_effect,
        'avg_effect': np.mean(treatment_effect)
    }

# 对单个处理单位应用
sc_result = simple_synthetic_control(df_nonparallel, treated_unit=0)

print("\n合成控制法结果 (单位 0):")
print("=" * 50)
print(f"各期处理效应: {sc_result['treatment_effect'].round(2)}")
print(f"平均处理效应: {sc_result['avg_effect']:.3f}")
print(f"真实效应: 5.0")

---

## Part 5: 完整诊断流程

### 🧪 TODO 练习: 实现 DID 诊断流程

In [ ]:
def did_diagnostic_pipeline(df, treatment_period=6):
    """
    完整的 DID 诊断流程
    """
    print("=" * 70)
    print("DID 诊断报告")
    print("=" * 70)
    
    # Step 1: 数据概览
    print("\n【Step 1: 数据概览】")
    n_units = df['unit'].nunique()
    n_treated = df[df['treated']==1]['unit'].nunique()
    n_control = df[df['treated']==0]['unit'].nunique()
    n_periods = df['period'].nunique()
    print(f"  总单位数: {n_units}")
    print(f"  处理组: {n_treated}, 对照组: {n_control}")
    print(f"  时期数: {n_periods}, 处理时点: {treatment_period}")
    
    # Step 2: 平行趋势检验
    print("\n【Step 2: 平行趋势检验】")
    pt_result = parallel_trend_test(df, treatment_period)
    if pt_result['reject_H0']:
        print(f"  ❌ 拒绝平行趋势假设 (p = {pt_result['p_value']:.4f})")
        print(f"     处理组额外趋势: {pt_result['coefficient']:.4f}/期")
        parallel_ok = False
    else:
        print(f"  ✅ 不拒绝平行趋势假设 (p = {pt_result['p_value']:.4f})")
        parallel_ok = True
    
    # Step 3: Anticipation Effect 检验
    print("\n【Step 3: Anticipation Effect 检验】")
    has_anticipation = diagnose_anticipation(df, treatment_period, n_pre_periods=2)
    
    # Step 4: DID 估计
    print("\n【Step 4: DID 估计】")
    
    # 标准 DID
    standard_coef, standard_se = did_estimate(df, treatment_period)
    print(f"  标准 DID: {standard_coef:.3f} (SE = {standard_se:.3f})")
    
    if not parallel_ok:
        # 组特定趋势 DID
        gt_result = did_with_group_trends(df, treatment_period)
        print(f"  组特定趋势 DID: {gt_result['did_coef']:.3f} (SE = {gt_result['did_se']:.3f})")
    
    # Step 5: 建议
    print("\n【Step 5: 建议】")
    if parallel_ok and not has_anticipation:
        print("  ✅ 平行趋势满足，无 Anticipation，可使用标准 DID")
    elif has_anticipation:
        print("  ⚠️ 存在 Anticipation Effect，建议:")
        print("     1. 调整处理时点")
        print("     2. 报告不同处理时点的敏感性分析")
    elif not parallel_ok:
        print("  ⚠️ 平行趋势不满足，建议:")
        print("     1. 使用组特定趋势 DID")
        print("     2. 考虑合成控制法")
        print("     3. 寻找更好的对照组")
    
    return {
        'parallel_ok': parallel_ok,
        'has_anticipation': has_anticipation,
        'standard_did': (standard_coef, standard_se)
    }

# 运行诊断
print("\n" + "#" * 70)
print("场景 1: 不满足平行趋势")
print("#" * 70)
did_diagnostic_pipeline(df_nonparallel)

---

## 💡 思考题

1. **检验功效**：平行趋势检验不拒绝 H0 就意味着平行趋势成立吗？
   - 提示：考虑样本量和检验功效

2. **多期处理**：如果不同单位在不同时期接受处理，如何检验平行趋势？
   - 提示：Staggered DID

3. **非线性趋势**：如果趋势是非线性的，组特定线性趋势 DID 还有效吗？
   - 提示：考虑更灵活的趋势控制

---

## 📝 总结

### 平行趋势诊断方法

| 方法 | 优点 | 缺点 |
|------|------|------|
| 可视化 | 直观 | 主观 |
| 事件研究 | 全面，可看每期 | 多重比较问题 |
| 统计检验 | 正式 | 功效可能不足 |

### 平行趋势违背的处理

| 情况 | 推荐方法 |
|------|----------|
| Anticipation | 调整处理时点 |
| 趋势差异（线性）| 组特定趋势 DID |
| 趋势差异（复杂）| 合成控制法 |

### 面试要点

- 能解释平行趋势假设的含义
- 知道如何用事件研究检验
- 能识别 Anticipation Effect
- 了解替代方法的适用场景